In [1]:
## A note on the below
## This is a monstrosity. I'm not proud of it. I was having fun with exec and eval.


###### NOTEBOOK INPUTS #####
dataset_names = ["CIFAR10"] #Need to include "CelebA", "CIFAR10", "MNIST"
model_names = ["VAE","BetaVAE","DVAE"]  #"VAE","BetaVAE",
metrics = ["FID"]
drive_folder = "~/data/10-708-project-results/trained_models/"       # the name of the google drive for saving
learning_rate=1e-3         # Need to get SOTA LR or do hyperparameter optimization
batch_size=100             # Same as LR
num_epochs=100             # Same as LR


In [2]:
!nvidia-smi

Thu Apr 14 13:54:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
|  0%   34C    P8    33W / 350W |    180MiB / 24234MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
for model_name in model_names:
  exec(f"from pythae.models import {model_name}, {model_name}Config")
from pythae.trainers import BaseTrainingConfig
from pythae.pipelines.training import TrainingPipeline
for dataset_name in dataset_names:
  exec(f"from pythae.models.nn.benchmarks.{''.join(i.lower() for i in dataset_name if not i.isdigit())} import Encoder_VAE_{''.join(i.upper() for i in dataset_name if not i.isdigit())}, Decoder_AE_{''.join(i.upper() for i in dataset_name if not i.isdigit())}")

In [4]:
import os

In [5]:
!nvidia-smi

Thu Apr 14 13:54:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
|  0%   34C    P8    28W / 350W |    180MiB / 24234MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
import torch
import torchvision.datasets as datasets


In [7]:
# loads each dataset in the list of dataset names
# maybe eval is bad but hopefully it works
for dataset_name in dataset_names:
  print(f"loading {dataset_name}")
  if dataset_name != "CelebA":
    trainset = eval(f"datasets.{dataset_name}(root=\'~/data\', train=True, download=True, transform=None)")
  else: #CelebA takes a different format
    trainset = eval(f"datasets.{dataset_name}(root=\'~/data\', split=\'train\', download=True, transform=None)")
  data = trainset.data
  num_train = int(data.shape[0] * 0.80)
  num_eval = data.shape[0] - num_train
  if len(data.shape) == 3:
    [H, W] = data.shape[-2:]
    data = data.reshape(-1, 1, H, W)
  assert(len(data.shape) == 4)
  if data.shape[-1] == 3:
    data = data.transpose(0,3,1,2)
  assert(data.max()>1)
  data = data / 255

  exec(f"{dataset_name}_train_dataset = data[:-num_eval]")
  exec(f"{dataset_name}_eval_dataset = data[-num_eval:]")
  


loading CIFAR10


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting /home/everett/data/cifar-10-python.tar.gz to /home/everett/data


In [8]:
#train all the models on all the datasets
for model_name in model_names:
  exec(f"""{model_name.lower()}_model_config = {model_name}Config(
      input_dim=(1, 28, 28),
      latent_dim=16
    )""")
  for dataset_name in dataset_names:
    training_config = BaseTrainingConfig(
      output_dir=f"{dataset_name}_{model_name}",
      learning_rate=learning_rate,        
      batch_size=batch_size,
      num_epochs=num_epochs,
    )
    print(f"dataset = {dataset_name}, model = {model_name}")

    exec(f"""{model_name.lower()}_{dataset_name.lower()} = {model_name}(
      model_config={model_name.lower()}_model_config,
      encoder=Encoder_VAE_{''.join(i.upper() for i in dataset_name if not i.isdigit())}({model_name.lower()}_model_config),
      decoder=Decoder_AE_{''.join(i.upper() for i in dataset_name if not i.isdigit())}({model_name.lower()}_model_config))""")

    exec(f"""{model_name.lower()}_{dataset_name.lower()}_pipeline = TrainingPipeline(
      training_config=training_config,
      model = {model_name.lower()}_{dataset_name.lower()}
    )""")

    exec(f"""{model_name.lower()}_{dataset_name.lower()}_pipeline(
      train_data={dataset_name}_train_dataset,
      eval_data={dataset_name}_eval_dataset
    )""")

Preprocessing train data...


dataset = CIFAR10, model = VAE


Preprocessing eval data...

Using Base Trainer

Created CIFAR10_VAE folder since did not exist.

Model passed sanity check !

Created CIFAR10_VAE/VAE_training_2022-04-14_13-54-36. 
Training config, checkpoints and final model will be saved here.

Successfully launched training !

Eval of epoch 1/100: 100%|██████████| 100/100 [00:01<00:00, 54.35batch/s]
----------------------------------------------------------------
Epoch 1: Train loss: 216.1320106506
Epoch 1: Eval loss: 120.1052214813
----------------------------------------------------------------
Eval of epoch 2/100: 100%|██████████| 100/100 [00:01<00:00, 54.81batch/s]
----------------------------------------------------------------
Epoch 2: Train loss: 109.5642706299
Epoch 2: Eval loss: 98.4661802673
----------------------------------------------------------------
Eval of epoch 3/100: 100%|██████████| 100/100 [00:01<00:00, 54.11batch/s]
----------------------------------------------------------------
Epoch 3: Train loss: 101.139556

Epoch 00051: reducing learning rate of group 0 to 5.0000e-04.


Eval of epoch 52/100: 100%|██████████| 100/100 [00:01<00:00, 52.04batch/s]
----------------------------------------------------------------
Epoch 52: Train loss: 71.5484412003
Epoch 52: Eval loss: 76.9380093384
----------------------------------------------------------------
Eval of epoch 53/100: 100%|██████████| 100/100 [00:01<00:00, 53.26batch/s]
----------------------------------------------------------------
Epoch 53: Train loss: 71.1429966927
Epoch 53: Eval loss: 77.5054271698
----------------------------------------------------------------
Eval of epoch 54/100: 100%|██████████| 100/100 [00:01<00:00, 53.38batch/s]
----------------------------------------------------------------
Epoch 54: Train loss: 70.9386919975
Epoch 54: Eval loss: 77.4544010925
----------------------------------------------------------------
Eval of epoch 55/100: 100%|██████████| 100/100 [00:01<00:00, 53.17batch/s]
----------------------------------------------------------------
Epoch 55: Train loss: 70.7200325

...

Eval of epoch 76/100: 100%|██████████| 100/100 [00:01<00:00, 50.71batch/s]
----------------------------------------------------------------
Epoch 76: Train loss: 66.3920766258
Epoch 76: Eval loss: 79.3524862671
----------------------------------------------------------------
Eval of epoch 77/100: 100%|██████████| 100/100 [00:01<00:00, 50.60batch/s]
----------------------------------------------------------------
Epoch 77: Train loss: 66.2693958378
Epoch 77: Eval loss: 79.345199585
----------------------------------------------------------------
Eval of epoch 78/100: 100%|██████████| 100/100 [00:01<00:00, 50.06batch/s]
----------------------------------------------------------------
Epoch 78: Train loss: 66.2625525856
Epoch 78: Eval loss: 80.0297328186
----------------------------------------------------------------
Eval of epoch 79/100: 100%|██████████| 100/100 [00:01<00:00, 50.54batch/s]
----------------------------------------------------------------
Epoch 79: Train loss: 66.25090382

Epoch 00086: reducing learning rate of group 0 to 3.1250e-05.


Eval of epoch 87/100: 100%|██████████| 100/100 [00:01<00:00, 50.70batch/s]
----------------------------------------------------------------
Epoch 87: Train loss: 65.8066354275
Epoch 87: Eval loss: 79.925329361
----------------------------------------------------------------
Eval of epoch 88/100: 100%|██████████| 100/100 [00:01<00:00, 50.59batch/s]
----------------------------------------------------------------
Epoch 88: Train loss: 65.8137796974
Epoch 88: Eval loss: 79.9140302277
----------------------------------------------------------------
Eval of epoch 89/100: 100%|██████████| 100/100 [00:01<00:00, 50.57batch/s]
----------------------------------------------------------------
Epoch 89: Train loss: 65.7662620926
Epoch 89: Eval loss: 79.8337981415
----------------------------------------------------------------
Eval of epoch 90/100: 100%|██████████| 100/100 [00:01<00:00, 50.68batch/s]
----------------------------------------------------------------
Epoch 90: Train loss: 65.74734311

Epoch 00097: reducing learning rate of group 0 to 1.5625e-05.


Eval of epoch 98/100: 100%|██████████| 100/100 [00:01<00:00, 50.64batch/s]
----------------------------------------------------------------
Epoch 98: Train loss: 65.5454702473
Epoch 98: Eval loss: 80.0062677002
----------------------------------------------------------------
Eval of epoch 99/100: 100%|██████████| 100/100 [00:01<00:00, 50.74batch/s]
----------------------------------------------------------------
Epoch 99: Train loss: 65.5406219196
Epoch 99: Eval loss: 79.8607242584
----------------------------------------------------------------
Eval of epoch 100/100: 100%|██████████| 100/100 [00:01<00:00, 50.70batch/s]
----------------------------------------------------------------
Epoch 100: Train loss: 65.5484293461
Epoch 100: Eval loss: 80.1279492188
----------------------------------------------------------------
----------------------------------
Training ended!
Saved final model in CIFAR10_DVAE/DVAE_training_2022-04-14_14-59-13/final_model
